In [ ]:
#Downloading the raw data from Amazon Web Service (AWS) S3 
#This is the cloud warehouse where we store data for use on the analytical platform

#This is a package that allows us to download data from AWS S3 in a more usable way
#You may need to install it in the Terminal --> pip install arrow-pd-parser
#Alternatively --> pip install --user arrow-pd-parser

In [ ]:
import boto3
import pandas as pd
import numpy as np
import io
import xlsxwriter
from openpyxl.utils.dataframe import dataframe_to_rows
import openpyxl
from openpyxl import load_workbook
from arrow_pd_parser import reader, writer

In [ ]:
#Location of data - setting up the folder and file names
bucket = "s3://alpha-piatool/"
input_folder = "sopdq_AV/inputs/"
output_folder = "sopdq_AV/outputs/"

In [ ]:
#Choose your dataset

valid_dates = ["test23", "Mar22", "Apr22", "May22", "Jun22", "Jul22", "Aug22", "Sep22", "Oct22", "Nov22", "Dec22", 
               "Jan23", "Feb23", "Mar23", "Apr23", "May23", "Jun23", "Jul23", "Aug23", "Sep23", "Oct23", "Nov23"]

while True:    
    date = input("Please choose the month you want an Exception Report for: ")

    if date not in valid_dates:
        print("That was not a valid choice. Please try again")
        continue 
    else:
        print("You have made a valid choice.")
        raw_data = "WFPT Staff Data " + date + ".csv"
        look_up_data = "WFPT lookups " + date + ".xlsx"
        print(raw_data)
        break       
        

In [ ]:
#Loading the raw data stored by David Yuen 
raw_data_file = reader.read(bucket+input_folder+raw_data, file_format="csv")

In [ ]:
print(bucket+input_folder+raw_data)

In [ ]:
raw_data_file.head(2)

In [ ]:
#Tracie Kilbey creates four sheets from the raw dataset, which she then uses for the exception reports
#Let's start with creating the first sheet, as it's the simplest - "WfPT all"
#This is just the raw data, but we'll make a few changes to column headings to make it easier to use

In [ ]:
WfPT_all = raw_data_file

In [ ]:
# WfPT_all.columns = WfPT_all.columns.str.replace(" ", "_")
# WfPT_all.columns = map(str.lower, WfPT_all.columns)

In [ ]:
raw_data_file.columns

In [ ]:
#Save this file in AWS S3 
writer.write(df=WfPT_all, output_path=bucket+output_folder+"WfPT all.csv", file_format="csv")

In [ ]:
#Completed first file
WfPT_all
len(WfPT_all)

In [ ]:
# Now create second file - Probation inc HQ
# Filter Column "NOMS-MOJ" to leave only "NPS" or "NOMS HQ"

In [ ]:
Probation_inc_HQ = raw_data_file[raw_data_file["NOMS-MOJ"].isin(["NOMS HQ", "NPS"])]

In [ ]:
#Save on AWS S3
writer.write(df=Probation_inc_HQ, output_path=bucket+output_folder+"Probation inc HQ.csv", file_format="csv")

In [ ]:
#Completed second file
Probation_inc_HQ.head(2)
len(Probation_inc_HQ)

In [ ]:
# Now create the third file - Probation some HQ
#For those working in Probation Delivery Units (PDUs) and Region Offices, 
#identify those who have NOMS-MOJ (column S) value of "NPS".
# Add in Other HQ staff – see establishment names below (use the Prog CC and Admin CC)
# Cost Centre = 10207830, 10207831, 10207812

In [ ]:
probation_staff = raw_data_file[raw_data_file["NOMS-MOJ"] == "NPS"] # Only include National Probation Service
probation_staff_exc_aps = probation_staff[~probation_staff["Establishment"].str.contains("APs")] # remove Approved Premises
probation_staff_exc_aps = probation_staff[~probation_staff["Area_Directorate"].str.contains("Approved Premises")] # remove Approved Premises
probation_staff_exc_aps = probation_staff_exc_aps[~probation_staff_exc_aps["Establishment"].str.contains("Approved Premises")]# remove Approved Premises
hq_staff = raw_data_file[raw_data_file["NOMS-MOJ"] == "NOMS HQ"] # Get cost centres from HQ
some_hq_staff = hq_staff[hq_staff["Cost Centre"].isin([10207830, 10207831, 10207812])] # Include only three cost centres from HQ

probation_some_hq = pd.concat([probation_staff_exc_aps,some_hq_staff]) # combine the National Probation staff (excluding Approved Premises) AND three cost centres from HQ

In [ ]:
probation_some_hq
len(probation_some_hq)

In [ ]:
#Save on AWS S3
writer.write(df=probation_some_hq, output_path=bucket+output_folder+"Probation some HQ.csv", file_format="csv")

In [ ]:
#Create the fourth sheet - APs
## Filter for NOMS-HQ field of "NOMS HQ" and then filter for all establishment with “APs" at end of name. 
## Filter for National Approved Premises and AP Professionalisation Project (as in the table below).
### Cost centres = 10207588, 10207586


In [ ]:
hq_staff_only_aps = raw_data_file[raw_data_file["Establishment"].str.contains("APs")]
other_aps = raw_data_file[raw_data_file["Cost Centre"].isin([10207588, 10207586])] 
# "AP Counter Terrorism and Drug Strategy" has not been included.
# David Yuen advised in email dated 30.11.2022 - "It is more of a HQ type unit so I wouldn’t include with the other Approved Premises"
aps = pd.concat([hq_staff_only_aps,other_aps])

In [ ]:
len(aps)

In [ ]:
#Save on AWS S3
writer.write(df=aps, output_path=bucket+output_folder+"APs.csv", file_format="csv")

In [ ]:
####################################################################################################################
# Next step - to combine csv files into one workbook on excel
####################################################################################################################

In [ ]:
## Name of csv files
# WfPT_all
# Probation_inc_HQ
# probation_some_hq
# aps

In [ ]:
with io.BytesIO() as output:
    with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
        WfPT_all.to_excel(writer, sheet_name='WfPT All')
        Probation_inc_HQ.to_excel(writer, sheet_name='Probation inc HQ')
        probation_some_hq.to_excel(writer, sheet_name='Probation some HQ')
        aps.to_excel(writer, sheet_name='APs')
    data = output.getvalue()
s3 = boto3.resource('s3')
file_location = "sopdq_AV/outputs/WFPT Staff Data " + date + " Probation.xlsx"
s3.Bucket('alpha-piatool').put_object(Key=file_location, Body=data)


In [ ]:
####################################################################################################################
# Next step - Add look up files
####################################################################################################################

In [ ]:
look_up_data = bucket+input_folder+look_up_data

In [ ]:
look_up_data

In [ ]:
# #Loading the look up file stored in reference folder
look_ups = pd.read_excel(bucket+input_folder+"lookups_may22.xlsx",
                         sheet_name="Lookups", skiprows=2)

cost_centre_consistency = pd.read_excel(bucket+input_folder+"Lookups for Probation DQ v1.4.xlsx", 
                                        sheet_name="Cost Centre types", skiprows=2)

In [ ]:
#Loading the look up file stored in reference folder
cost_centres = pd.read_excel(look_up_data, sheet_name="Cost Centre Lookup")

hr_locations = pd.read_excel(look_up_data, sheet_name="HR Location Lookup")

int_functions = pd.read_excel(look_up_data, sheet_name="Internal Function Lookup")

jobs = pd.read_excel(look_up_data, sheet_name="Job Title Lookup")

In [ ]:
################################################################################################################
# Main Data Quality checks 
## Cost Centres
## HR locations
## Internal functions
## Job (Title)
## *Job (Band) not required but added in any event
################################################################################################################

In [ ]:
#Tidy up Cost Centre lookup column titles
cost_centre_lookup = cost_centres[['Cost Centre Number', 'Cost Centre Number']].copy(deep=True)
cost_centre_lookup.columns = ['Cost Centre', 'Cost Centre Number']
cost_centre_lookup

In [ ]:
#Tidy up HR Location lookup column titles
hr_location_lookup = hr_locations[["Location Name"]].copy(deep=True)
hr_location_lookup["HR Valid"] = "Valid"
hr_location_lookup.columns = ["HR Location", "HR Valid"]
hr_location_lookup = hr_location_lookup.dropna().drop_duplicates()
hr_location_lookup

In [ ]:
#Tidy up Internal Function lookup column title
internal_function_lookup = int_functions[['Internal Function', 'Internal Function Valid']].copy(deep=True)
internal_function_lookup

In [ ]:
len(internal_function_lookup)

In [ ]:
#Tidy up Job (title) lookup column title
job_title_lookup = jobs[["Job", "Job Function Valid"]].copy(deep=True)
job_title_lookup = job_title_lookup.dropna().drop_duplicates()
job_title_lookup

In [ ]:
# #Tidy up Job (band) lookup column title
# new_header = job_band_lookup.iloc[0] #grab the first row for the header
# job_band_lookup = job_band_lookup[1:] #take the data less the header row
# job_band_lookup.columns = new_header #set the header row as the df header

In [ ]:
# job_band_lookup = look_ups[['Grade DY']]
# job_band_lookup.head(3)

In [ ]:
cost_centre_consistency = cost_centre_consistency[["Cost Centre", "Cost Centre Type"]]
cost_centre_consistency["Cost Centre"]=cost_centre_consistency["Cost Centre"].astype("Int64")
cost_centre_consistency.head(3)

In [ ]:
# Save all lookups in single excel file
with io.BytesIO() as output:
    with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
        cost_centre_lookup.to_excel(writer, sheet_name='Cost Centre Lookup')
        internal_function_lookup.to_excel(writer, sheet_name='Internal Function Lookup')
        hr_location_lookup.to_excel(writer, sheet_name='HR Location Lookup')
        job_title_lookup.to_excel(writer, sheet_name='Job Title Lookup')
        cost_centre_consistency.to_excel(writer, sheet_name="Cost Centre type")
    data = output.getvalue()
s3 = boto3.resource('s3')
s3.Bucket('alpha-piatool').put_object(Key='sopdq_AV/outputs/WFPT lookups.xlsx', Body=data)


In [ ]:
################################################################################
# Workforce Probation Error Report
################################################################################

In [ ]:
## Step 0: Create Sheet "WfPT Probation Staff in Post"
### Use Probation Some HQ 
### Use cost_centre_consistency

In [ ]:
len(probation_some_hq)

In [ ]:
probation_some_hq.columns

In [ ]:
wfpt_probation_sip = pd.merge(probation_some_hq, cost_centre_consistency, on = "Cost Centre", how = "left")

In [ ]:
cols = ['Cost Centre Type', 'Internal Function', 'Job']
wfpt_probation_sip["Joined CCType,IF, Job For Consistency Check"] = wfpt_probation_sip[cols].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)

In [ ]:
##################################################################################################################

In [ ]:
## Step 1: Create "Cost Centre Error Flag" column

# Merge main to lookup on cost centre
# If matched, "Valid", 
# If NA, "Missing", 
# If not NA and not matched, "Invalid"

In [ ]:
wfpt_probation_sip2 = pd.merge(wfpt_probation_sip, cost_centre_lookup, on="Cost Centre", how="left")

In [ ]:
wfpt_probation_sip2["Cost Centre Error Flag"] = np.where(wfpt_probation_sip2["Cost Centre Number"] == wfpt_probation_sip2["Cost Centre"], "Valid", 
                                                         np.where(wfpt_probation_sip2["Cost Centre"].isna(), "Missing", "Invalid"))

In [ ]:
print(
    len(wfpt_probation_sip2[wfpt_probation_sip2["Cost Centre Error Flag"]=="Invalid"]), 
    len(wfpt_probation_sip2[wfpt_probation_sip2["Cost Centre Error Flag"]=="Valid"]), 
    len(wfpt_probation_sip2[wfpt_probation_sip2["Cost Centre Error Flag"]=="Missing"]), 
    len(wfpt_probation_sip2)
)

In [ ]:
## Step 2: Create "HR Location Error Flag" column

# Merge main to lookup on cost centre
# If matched, "Valid", 
# If NA, "Missing", 
# If not NA and not matched, "Invalid"

In [ ]:
wfpt_probation_sip3 = pd.merge(wfpt_probation_sip2, hr_location_lookup, on="HR Location", how="left")

In [ ]:
wfpt_probation_sip3["HR Location Error Flag"] = np.where(wfpt_probation_sip3["HR Location"].isnull(), "Missing", 
                                                         np.where(wfpt_probation_sip3["HR Valid"].isnull(), "Invalid", "Valid"))

In [ ]:
print(
    r"Invalid total: ", len(wfpt_probation_sip3[wfpt_probation_sip3["HR Location Error Flag"]=="Invalid"]), "\n",
    r"Valid total: ", len(wfpt_probation_sip3[wfpt_probation_sip3["HR Location Error Flag"]=="Valid"]), "\n", 
    r"Missing Total: ", len(wfpt_probation_sip3[wfpt_probation_sip3["HR Location Error Flag"]=="Missing"]), "\n",
    r"Total Staff:", len(wfpt_probation_sip3)
)

In [ ]:
## Step 3: Create "Internal Function Error Flag" column

# Merge main to lookup on cost centre
# If matched, "Valid", 
# If NA, "Missing", 
# If not NA and not matched, "Invalid"

In [ ]:
wfpt_probation_sip4 = pd.merge(wfpt_probation_sip3, internal_function_lookup, on="Internal Function", how="left")

In [ ]:
wfpt_probation_sip4["Internal Function Error Flag"] = np.where(wfpt_probation_sip4["Internal Function"].isnull(), "Missing", 
                                                         np.where(wfpt_probation_sip4["Internal Function Valid"].isnull(), "Invalid", "Valid"))

In [ ]:
print(
    r"Invalid total: ", len(wfpt_probation_sip4[wfpt_probation_sip4["Internal Function Error Flag"]=="Invalid"]), "\n",
    r"Valid total: ", len(wfpt_probation_sip4[wfpt_probation_sip4["Internal Function Error Flag"]=="Valid"]), "\n", 
    r"Missing Total: ", len(wfpt_probation_sip4[wfpt_probation_sip4["Internal Function Error Flag"]=="Missing"]), "\n",
    r"Total Staff:", len(wfpt_probation_sip4)
)

In [ ]:
## Step 4: Create "Job Error Flag" column

# Merge main to lookup on cost centre
# If matched, "Valid", 
# If NA, "Missing", 
# If not NA and not matched, "Invalid"

In [ ]:
job_title_lookup.head(2)

In [ ]:
wfpt_probation_sip5 = pd.merge(wfpt_probation_sip4, job_title_lookup, on="Job", how="left")

In [ ]:
wfpt_probation_sip5.head(2)

In [ ]:
wfpt_probation_sip5["Job Error Flag"] = np.where(wfpt_probation_sip5["Job"].isnull(), "Missing",
                                                 np.where(wfpt_probation_sip5["Job Function Valid"].isnull(), "Invalid", "Valid"))

In [ ]:
print(
    r"Invalid total: ", len(wfpt_probation_sip5[wfpt_probation_sip5["Job Error Flag"]=="Invalid"]), "\n",
    r"Valid total: ", len(wfpt_probation_sip5[wfpt_probation_sip5["Job Error Flag"]=="Valid"]), "\n", 
    r"Missing Total: ", len(wfpt_probation_sip5[wfpt_probation_sip5["Job Error Flag"]=="Missing"]), "\n",
    r"Total Staff:", len(wfpt_probation_sip5)
)

In [ ]:
wfpt_probation_sip5.head(1)

In [ ]:
wfpt_probation_sip5.columns

In [ ]:
wfpt_probation_sip_final = wfpt_probation_sip5.drop(["Cost Centre Number", "HR Valid", "Internal Function Valid", "Job Function Valid"], axis=1)

In [ ]:
wfpt_probation_sip_final.head(2)

In [ ]:
len(wfpt_probation_sip_final)

In [ ]:
####################################################################################################################
# Final Error Report
####################################################################################################################

In [ ]:
#################################################################
# Step 1: WfPT data
#################################################################

In [ ]:
# WORKFORCE PLANNING TOOL DATA
# Employee Number
# Cost Centre
# Cost Centre Description (Establishment)
# HR Location
# Internal Function
# Grade
# Job (title)
# FTE
# Location Postcode
# Area_Directorate
# Hours

# ERROR FLAGS
# Cost Centre Error Flag
# HR Location Error Flag
# Internal Function Error Flag
# Job Error Flag

In [ ]:
wfpt_data = wfpt_probation_sip_final[["Employee Number", "Cost Centre", "Establishment", "HR Location", "Internal Function", 
                                      "Grade", "Job", "FTE", "Location Post Code", "Area_Directorate", "Hours", 
                                     'Cost Centre Error Flag', 'HR Location Error Flag','Internal Function Error Flag', 'Job Error Flag']]

In [ ]:
wfpt_probation_sip_final.columns

In [ ]:
wfpt_data.head(1)

In [ ]:
#################################################################
# Step 2: Tables % Valid
#################################################################

In [ ]:
#################################################################
# Rename columns
#################################################################

In [ ]:
wfpt_data_report = wfpt_data.copy(deep=True)

In [ ]:
wfpt_data_report.rename(columns={"Establishment" : 'Cost Centre Description (Establishment)', "Job" : 'Job (title)'}, inplace=True)

In [ ]:
wfpt_data_report.head(1)

In [ ]:
#Columns: 
## Region (Area Directorate)
## Total Cases
### Number missing, Number invalid, Number valid
## Cost Centre 
### Number missing, Number invalid, Number valid
## HR Location
### Number missing, Number invalid, Number valid
## Internal Function
### Number missing, Number invalid, Number valid
## Job
### Number missing, Number invalid, Number valid
## Total Valid 
### Number missing, Number invalid, Number valid
## Percentage point difference from England and Wales
## Comparison with England and Wales
## Trend June 2021 to March 2022

In [ ]:
#################################################################
#Table 1: Number of errors and percentage of valid cases by Region and variable 
# (excluding Approved Premises in England and some HQ functions), 
# 31 March 2022 with trends
################################################################## 

In [ ]:
table1 = wfpt_data_report[["Area_Directorate", "Cost Centre Error Flag", "HR Location Error Flag", 
                          "Internal Function Error Flag", "Job Error Flag"]]

In [ ]:
#group by area directorate
#sum total cases 

In [ ]:
table1.head(3)

In [ ]:
#group by area directorate
#Cost Centre: sum missing, invalid, valid, % of valid/total cases

In [ ]:
cols = ['Cost Centre Error Flag', 'HR Location Error Flag', 'Internal Function Error Flag', 'Job Error Flag']
table1_valid_raw = []

for col in cols:
    df = pd.crosstab(table1['Area_Directorate'], table1[col])
    df.columns = pd.MultiIndex.from_product([[col], df.columns.tolist()])
    table1_valid_raw.append(df)

table1_valid = pd.concat(table1_valid_raw, axis=1)

In [ ]:
table1_valid

In [ ]:
complete_list = [('Cost Centre Error Flag', 'Invalid'),
                 ('Cost Centre Error Flag', 'Valid'),
                 ('Cost Centre Error Flag', 'Missing'),
                 ('HR Location Error Flag', 'Invalid'),
                 ('HR Location Error Flag', 'Missing'),
                 ('HR Location Error Flag', 'Valid'),
                 ('Internal Function Error Flag', 'Invalid'),
                 ('Internal Function Error Flag', 'Missing'),
                 ('Internal Function Error Flag', 'Valid'),
                 ('Job Error Flag', 'Invalid'),
                 ('Job Error Flag', 'Missing'),
                 ('Job Error Flag', 'Valid')]

In [ ]:
missing_items_table1 = list(set(complete_list) - set(table1_valid.columns.to_list()))

In [ ]:
for item in missing_items_table1:
    table1_valid[item] = 0
    table1_valid = table1_valid.sort_index(axis=1)

In [ ]:
table1_valid.head(1)

In [ ]:
table1_valid[("Total Cases", "")] = (table1_valid[("Cost Centre Error Flag", "Invalid")] + 
                                     table1_valid[("Cost Centre Error Flag", "Valid")] + 
                                     table1_valid[("Cost Centre Error Flag", "Missing")])

In [ ]:
table1_valid.head(3)

In [ ]:
table1_valid[("Cost Centre Error Flag", "% valid")] = table1_valid[("Cost Centre Error Flag", "Valid")] / table1_valid[("Total Cases", "")]

table1_valid[("HR Location Error Flag", "% valid")] = table1_valid[("HR Location Error Flag", "Valid")] /table1_valid[("Total Cases", "")]

table1_valid[("Internal Function Error Flag", "% valid")] = table1_valid[("Internal Function Error Flag", "Valid")] /table1_valid[("Total Cases", "")]

table1_valid[("Job Error Flag", "% valid")] =table1_valid[("Job Error Flag", "Valid")] /table1_valid[("Total Cases", "")]

In [ ]:
table1_valid = table1_valid.sort_index(axis=1)

In [ ]:
#formatting for template
table1_valid[""] = ""
table1_valid[" "] = " "

In [ ]:
table1_valid[("Total Valid", "Missing")] = (table1_valid[("Cost Centre Error Flag", "Missing")] +
                                                  table1_valid[("HR Location Error Flag", "Missing")] +
                                                  table1_valid[("Internal Function Error Flag", "Missing")] +
                                                  table1_valid[("Job Error Flag", "Missing")])

table1_valid[("Total Valid", "Invalid")] = (table1_valid[("Cost Centre Error Flag", "Invalid")] +
                                                  table1_valid[("HR Location Error Flag", "Invalid")] +
                                                  table1_valid[("Internal Function Error Flag", "Invalid")] +
                                                  table1_valid[("Job Error Flag", "Invalid")])

table1_valid[("Total Valid", "Valid")] = (table1_valid[("Cost Centre Error Flag", "Valid")] +
                                                  table1_valid[("HR Location Error Flag", "Valid")] +
                                                  table1_valid[("Internal Function Error Flag", "Valid")] +
                                                  table1_valid[("Job Error Flag", "Valid")])

In [ ]:
england_and_wales = (sum(table1_valid[("Total Valid", "Valid")]) / 
                     (sum(table1_valid[("Total Valid", "Missing")]) + 
                      sum(table1_valid[("Total Valid", "Invalid")]) + 
                      sum(table1_valid[("Total Valid", "Valid")])))
england_and_wales 

In [ ]:
# table1_valid[("Total Valid", "% valid")] = (table1_valid[("Total Valid", "Valid")] / 
#                                                   (table1_valid[("Total Valid", "Valid")] + 
#                                                    table1_valid[("Total Valid", "Invalid")] + 
#                                                    table1_valid[("Total Valid", "Missing")]))

In [ ]:
table1_valid[("Total Valid", "% valid")] = (table1_valid[("Total Valid", "Valid")] / 
                                            (table1_valid[("Total Cases", "")]*4))

In [ ]:
table1_valid[("Percentage point difference from England and Wales", "")] = (table1_valid[("Total Valid", "% valid")] - england_and_wales)*100

In [ ]:
table1_final = table1_valid.copy(deep = True)

In [ ]:
table1_final.columns

In [ ]:
table1_final = table1_final[[
            (                                                  '',        ''),
            (                                                 ' ',        ''),
            (                                       'Total Cases',        ''),
    
            (                            'Cost Centre Error Flag', 'Missing'),
            (                            'Cost Centre Error Flag', 'Invalid'),
            (                            'Cost Centre Error Flag',   'Valid'),
            (                            'Cost Centre Error Flag', '% valid'),

            (                            'HR Location Error Flag', 'Missing'),
            (                            'HR Location Error Flag', 'Invalid'),
            (                            'HR Location Error Flag',   'Valid'),
            (                            'HR Location Error Flag', '% valid'),
 
            (                      'Internal Function Error Flag', 'Missing'),
            (                      'Internal Function Error Flag', 'Invalid'),
            (                      'Internal Function Error Flag',   'Valid'),
            (                      'Internal Function Error Flag', '% valid'),
    
            (                                    'Job Error Flag', 'Missing'),
            (                                    'Job Error Flag', 'Invalid'),
            (                                    'Job Error Flag',   'Valid'),
            (                                    'Job Error Flag', '% valid'),
    
            (                                       'Total Valid', 'Missing'),
            (                                       'Total Valid', 'Invalid'),
            (                                       'Total Valid',   'Valid'),
            (                                       'Total Valid', '% valid'),
    
            ('Percentage point difference from England and Wales',        '')]]

In [ ]:
table1_final

In [ ]:
###########################################################################
# Table 2: Number of errors and percentage of valid cases by Region and variable 
# (excluding Approved Premises in England and some HQ functions), 
# 31 March 2022 with trends
############################################################################

In [ ]:
wfpt_data_report.columns

In [ ]:
table2 = wfpt_data_report[["Area_Directorate", "Cost Centre Description (Establishment)", "Cost Centre", 
                           "Cost Centre Error Flag", "HR Location Error Flag", "Internal Function Error Flag", "Job Error Flag"]]

In [ ]:
table2.head()

In [ ]:
cols = ['Cost Centre Error Flag', 'HR Location Error Flag', 'Internal Function Error Flag', 'Job Error Flag']
table2_valid_raw = []

for col in cols:
    df = pd.crosstab(table2["Cost Centre"], table2[col])
    df.columns = pd.MultiIndex.from_product([[col], df.columns.tolist()])
    table2_valid_raw.append(df)

table2_valid = pd.concat(table2_valid_raw, axis=1)

In [ ]:
missing_items_table2 = list(set(complete_list) - set(table2_valid.columns.to_list()))

In [ ]:
for item in missing_items_table2:
    table2_valid[item] = 0
    table2_valid = table2_valid.sort_index(axis=1)

In [ ]:
table2_valid

In [ ]:
table2_valid[("Total Cases", "")] = (table2_valid[("Cost Centre Error Flag", "Invalid")] + 
                                     table2_valid[("Cost Centre Error Flag", "Valid")] + 
                                     table2_valid[("Cost Centre Error Flag", "Missing")])

In [ ]:
table2_valid[("Cost Centre Error Flag", "% valid")] = table2_valid[("Cost Centre Error Flag", "Valid")] / table2_valid[("Total Cases", "")]

table2_valid[("HR Location Error Flag", "% valid")] = table2_valid[("HR Location Error Flag", "Valid")] /table2_valid[("Total Cases", "")]

table2_valid[("Internal Function Error Flag", "% valid")] = table2_valid[("Internal Function Error Flag", "Valid")] /table2_valid[("Total Cases", "")]

table2_valid[("Job Error Flag", "% valid")] = table2_valid[("Job Error Flag", "Valid")] /table2_valid[("Total Cases", "")]

In [ ]:
table2_valid = table2_valid.sort_index(axis=1)

In [ ]:
table2_valid[("Total Valid", "Missing")] = (table2_valid[("Cost Centre Error Flag", "Missing")] +
                                                  table2_valid[("HR Location Error Flag", "Missing")] +
                                                  table2_valid[("Internal Function Error Flag", "Missing")] +
                                                  table2_valid[("Job Error Flag", "Missing")])

table2_valid[("Total Valid", "Invalid")] = (table2_valid[("Cost Centre Error Flag", "Invalid")] +
                                                  table2_valid[("HR Location Error Flag", "Invalid")] +
                                                  table2_valid[("Internal Function Error Flag", "Invalid")] +
                                                  table2_valid[("Job Error Flag", "Invalid")])

table2_valid[("Total Valid", "Valid")] = (table2_valid[("Cost Centre Error Flag", "Valid")] +
                                                  table2_valid[("HR Location Error Flag", "Valid")] +
                                                  table2_valid[("Internal Function Error Flag", "Valid")] +
                                                  table2_valid[("Job Error Flag", "Valid")])

In [ ]:
england_and_wales = (sum(table2_valid[("Total Valid", "Valid")]) / 
                     (sum(table2_valid[("Total Valid", "Missing")]) + 
                      sum(table2_valid[("Total Valid", "Invalid")]) + 
                      sum(table2_valid[("Total Valid", "Valid")])))
england_and_wales 

In [ ]:
table2_valid[("Total Valid", "% valid")] = (table2_valid[("Total Valid", "Valid")] / 
                                            (table2_valid[("Total Cases", "")]*4))
                                                  

In [ ]:
table2_valid.iloc[5]

In [ ]:
table2_details = table2[["Area_Directorate", "Cost Centre", "Cost Centre Description (Establishment)"]]
table2_details = table2_details.drop_duplicates(ignore_index = True)
table2_details = table2_details.set_index("Cost Centre")
table2_details.columns = [table2_details.columns, ["", "Establishment"]]
table2_details

In [ ]:
table2_final = table2_valid.join(table2_details)
table2_final.reset_index()

In [ ]:
table2_final.head(2)

In [ ]:
table2_final_version = table2_final.reset_index()

In [ ]:
table2_final_version = table2_final_version.set_index("Area_Directorate")

In [ ]:
table2_final_version.columns

In [ ]:
table2_final_version = table2_final_version[[("Cost Centre", ""),
            ('Cost Centre Description (Establishment)', 'Establishment'),
            ('Total Cases',        ''),
            ('Cost Centre Error Flag', 'Missing'),
            ('Cost Centre Error Flag', 'Invalid'),
            ('Cost Centre Error Flag',   'Valid'),
            ('Cost Centre Error Flag', '% valid'),
            
            ('HR Location Error Flag', 'Missing'),
            ('HR Location Error Flag', 'Invalid'),
            ('HR Location Error Flag',   'Valid'),
            ('HR Location Error Flag', '% valid'),
                                             

            ('Internal Function Error Flag', 'Missing'),
            ('Internal Function Error Flag', 'Invalid'),
            ('Internal Function Error Flag',   'Valid'),
            ('Internal Function Error Flag', '% valid'),
                                             
            ('Job Error Flag', 'Missing'),
            ('Job Error Flag', 'Invalid'),
            ('Job Error Flag',   'Valid'),
            ('Job Error Flag', '% valid'),
                                             
            ('Total Valid', 'Missing'),
            ('Total Valid', 'Invalid'),
            ('Total Valid',   'Valid'),
            ('Total Valid', '% valid')]]

In [ ]:
table2_final_version[("Difference of % valid from England and Wales", "")] = round((table2_final_version[("Total Valid", "% valid")] - england_and_wales)*100,1)
table2_final_version[("Total Valid", "% valid")] = round(table2_final_version[("Total Valid", "% valid")],3)

In [ ]:
table2_final = table2_final_version.copy(deep = True).sort_values(by = ['Area_Directorate', 'Cost Centre'], na_position = 'first')

In [ ]:
table2_final.head(6)

In [ ]:
#################################################################
#Table 3: Number and percentage of complete cases by Region and variable 
#(excluding Approved Premises in England and some HQ functions), 31 March 2022
################################################################## 

In [ ]:
table1_valid.head(1)

In [ ]:
# Table 3 is just Table 1 with invalid and valid treated as complete. 
table3_complete = table1_valid.copy(deep = True)

In [ ]:
table3_complete[("Cost Centre Error Flag", "Complete")] = (table3_complete[("Cost Centre Error Flag", "Valid")] +
                                                           table3_complete[("Cost Centre Error Flag", "Invalid")]) 
                                                                    

table3_complete[("HR Location Error Flag", "Complete")] = (table3_complete[("HR Location Error Flag", "Valid")] + 
                                                             table3_complete[("HR Location Error Flag", "Invalid")])

table3_complete[("Internal Function Error Flag", "Complete")] = (table3_complete[("Internal Function Error Flag", "Valid")] +
                                                                 table3_complete[("Internal Function Error Flag", "Invalid")])

table3_complete[("Job Error Flag", "Complete")] = (table3_complete[("Job Error Flag", "Valid")] +
                                                   table3_complete[("Job Error Flag", "Invalid")])

In [ ]:
table3_complete.head(1)

In [ ]:
table3_complete[("Cost Centre Error Flag", "% complete")] = (table3_complete[("Cost Centre Error Flag", "Complete")] /
                                                                       table3_complete[("Total Cases", "")])

table3_complete[("HR Location Error Flag", "% complete")] = (table3_complete[("HR Location Error Flag", "Complete")] /
                                                                       table3_complete[("Total Cases", "")])

table3_complete[("Internal Function Error Flag", "% complete")] = (table3_complete[("Internal Function Error Flag", "Complete")] /
                                                                             table3_complete[("Total Cases", "")])

table3_complete[("Job Error Flag", "% complete")] = (table3_complete[("Job Error Flag", "Complete")] /
                                                               table3_complete[("Total Cases", "")])


In [ ]:
table3_complete

In [ ]:
table3_complete[("Total Complete", "Missing")] = (table3_complete[("Cost Centre Error Flag", "Missing")] + 
                                                  table3_complete[("HR Location Error Flag", "Missing")] + 
                                                  table3_complete[("Internal Function Error Flag", "Missing")] + 
                                                  table3_complete[("Job Error Flag", "Missing")])

table3_complete[("Total Complete", "Complete")] = (table3_complete[("Cost Centre Error Flag", "Complete")] + 
                                                  table3_complete[("HR Location Error Flag", "Complete")] + 
                                                  table3_complete[("Internal Function Error Flag", "Complete")] + 
                                                  table3_complete[("Job Error Flag", "Complete")])

table3_complete[("Total Complete", "% complete")] = ( (table3_complete[("Total Complete", "Complete")]) /
                                                    (table3_complete[("Total Complete", "Missing")] + table3_complete[("Total Complete", "Complete")]))

In [ ]:
#formatting for template
table3_complete[""] = ""
table3_complete[" "] = " "

In [ ]:
table3_complete

In [ ]:
table3_complete.columns

In [ ]:
table3_final = table3_complete[[(                                                  '', ''),
                                (                                                 ' ', ''),
                                ('Total Cases', ""),
                                ('Cost Centre Error Flag', "Missing"),
                                ('Cost Centre Error Flag', "Complete"),
                                ('Cost Centre Error Flag', "% complete"),
                                
                                ('HR Location Error Flag', "Missing"),
                                ('HR Location Error Flag', "Complete"),
                                ('HR Location Error Flag', "% complete"),
                                
                                ('Internal Function Error Flag', "Missing"),
                                ('Internal Function Error Flag', "Complete"),
                                ('Internal Function Error Flag', "% complete"),
                                
                                ('Job Error Flag', "Missing"),
                                ('Job Error Flag', "Complete"),
                                ('Job Error Flag', "% complete"),
                                
                                ('Total Complete', "Missing"),
                                ('Total Complete', "Complete"),
                                ('Total Complete', "% complete")]]

In [ ]:
#################################################################
#Table 4: Number and percentage of complete cases by Region, variable and Cost Centre 
#(excluding Approved Premises in England and some HQ functions), 31 March 2022
################################################################## 

In [ ]:
table2_final_version.head(1)

In [ ]:
table4_complete = table2_final_version.copy(deep=True)

In [ ]:
table4_complete.head(1)

In [ ]:
table4_complete[("Cost Centre Error Flag", "Complete")] = (table4_complete[("Cost Centre Error Flag", "Valid")] +
                                                           table4_complete[("Cost Centre Error Flag", "Invalid")]) 
                                                                    
table4_complete[("HR Location Error Flag", "Complete")] = (table4_complete[("HR Location Error Flag", "Valid")] + 
                                                             table4_complete[("HR Location Error Flag", "Invalid")])

table4_complete[("Internal Function Error Flag", "Complete")] = (table4_complete[("Internal Function Error Flag", "Valid")] +
                                                                 table4_complete[("Internal Function Error Flag", "Invalid")])

table4_complete[("Job Error Flag", "Complete")] = (table4_complete[("Job Error Flag", "Valid")] +
                                                   table4_complete[("Job Error Flag", "Invalid")])

In [ ]:
table4_complete[("Cost Centre Error Flag", "% complete")] = (table4_complete[("Cost Centre Error Flag", "Complete")] /
                                                                       table4_complete[("Total Cases", "")])

table4_complete[("HR Location Error Flag", "% complete")] = (table4_complete[("HR Location Error Flag", "Complete")] /
                                                                       table4_complete[("Total Cases", "")])

table4_complete[("Internal Function Error Flag", "% complete")] = (table4_complete[("Internal Function Error Flag", "Complete")] /
                                                                             table4_complete[("Total Cases", "")])

table4_complete[("Job Error Flag", "% complete")] = (table4_complete[("Job Error Flag", "Complete")] /
                                                               table4_complete[("Total Cases", "")])


In [ ]:
table4_complete[("Total Complete", "Missing")] = (table4_complete[("Cost Centre Error Flag", "Missing")] + 
                                                  table4_complete[("HR Location Error Flag", "Missing")] + 
                                                  table4_complete[("Internal Function Error Flag", "Missing")] + 
                                                  table4_complete[("Job Error Flag", "Missing")])

table4_complete[("Total Complete", "Complete")] = (table4_complete[("Cost Centre Error Flag", "Complete")] + 
                                                  table4_complete[("HR Location Error Flag", "Complete")] + 
                                                  table4_complete[("Internal Function Error Flag", "Complete")] + 
                                                  table4_complete[("Job Error Flag", "Complete")])

table4_complete[("Total Complete", "% complete")] = ((table4_complete[("Total Complete", "Complete")] /
                                                    (table4_complete[("Total Complete", "Missing")] + 
                                                     table4_complete[("Total Complete", "Complete")])))

In [ ]:
table4_complete.head(1)

In [ ]:
table4_final = table4_complete[[("Cost Centre", ""), 
                                ("Cost Centre Description (Establishment)", "Establishment"),
                                ('Total Cases', ""),
                                ('Cost Centre Error Flag', "Missing"),
                                ('Cost Centre Error Flag', "Complete"),
                                ('Cost Centre Error Flag', "% complete"),
                                
                                ('HR Location Error Flag', "Missing"),
                                ('HR Location Error Flag', "Complete"),
                                ('HR Location Error Flag', "% complete"),
                                
                                ('Internal Function Error Flag', "Missing"),
                                ('Internal Function Error Flag', "Complete"),
                                ('Internal Function Error Flag', "% complete"),
                                
                                ('Job Error Flag', "Missing"),
                                ('Job Error Flag', "Complete"),
                                ('Job Error Flag', "% complete"),
                                
                                ('Total Complete', "Missing"),
                                ('Total Complete', "Complete"),
                                ('Total Complete', "% complete")]].sort_values(by = ['Area_Directorate', 'Cost Centre'], na_position = 'first')

In [ ]:
table4_final.head(5)

In [ ]:
#################################################################
#AP Table: Approved Premises in England only
################################################################## 

In [ ]:
###################
#AP Lookup files
###################

In [ ]:
bucket+input_folder+"lookups_aps_" + date + ".csv"

In [ ]:
#Loading the look up file stored in reference folder
look_ups_aps = pd.read_csv(bucket+input_folder+"lookups_aps_" + date + ".csv", skiprows=1)

In [ ]:
look_ups_aps.head(1)

In [ ]:
#Cost Centre
cost_centre_lookup_aps = look_ups_aps[['Cost Centre', 'Cost Centre Description (Establishment)', 'Reporting','Date Open', 'Date closed']].iloc[0:13,:]
cost_centre_lookup_aps["Cost Centre"] = cost_centre_lookup_aps["Cost Centre"].astype(int) # converts cost centre number to integer format

#HR Location columns taken, and any duplicate values are removed
hr_location_lookup_aps = look_ups_aps[['HR Location', 'Reporting.3']].drop_duplicates(keep="first")
hr_location_lookup_aps = hr_location_lookup_aps[hr_location_lookup_aps["Reporting.3"] == "Valid"]

#IF columns taken, and any duplicate values are removed
internal_function_lookup_aps = look_ups_aps[['Internal Function', 'Reporting.1']].iloc[0:13,:].drop_duplicates(subset = "Internal Function", keep="first")

#Job Title columns taken, and any duplicate values are removed
job_title_lookup_aps = look_ups_aps[['Job (title)','Reporting.2']].drop_duplicates(keep="first").dropna()
job_title_lookup_aps = job_title_lookup_aps[job_title_lookup_aps['Reporting.2'] == "Valid"]
job_title_lookup_aps.rename(columns={'Job (title)': 'Job'}, inplace=True)

In [ ]:
cost_centre_lookup_aps

In [ ]:
# Alternative approach to jobs - link by job and grade
## job_title_lookup_aps = look_ups_aps[["Grade2",'Job (title)', 'Reporting.2', 'Date opened.1', 'Date closed.2']].iloc[0:32, :].drop_duplicates(keep="first").dropna()
## job_title_lookup_aps.rename(columns={'Grade2': 'Grade'}, inplace=True)

In [ ]:
# Save all Approved Premises lookups in single excel file
with io.BytesIO() as output:
    with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
        cost_centre_lookup_aps.to_excel(writer, sheet_name='Cost Centre Lookup')
        internal_function_lookup_aps.to_excel(writer, sheet_name='Internal Function Lookup')
        hr_location_lookup_aps.to_excel(writer, sheet_name='HR Location Lookup')
        job_title_lookup_aps.to_excel(writer, sheet_name='Job Title Lookup')
    data = output.getvalue()
s3 = boto3.resource('s3')
s3.Bucket('alpha-piatool').put_object(Key='sopdq_AV/outputs/Approved Premises lookups.xlsx', Body=data)

In [ ]:
############################################################################
#APs flagged for cost centre, job, hr location, internal function
############################################################################

In [ ]:
aps_final = aps[['Employee Number', 'Cost Centre', 'Establishment', 'HR Location', 'Internal Function', 'Grade', 'Job',
                 'FTE', 'Location Post Code', 'Area_Directorate', 'Hours']]

In [ ]:
#COST CENTRE
aps_final2 = pd.merge(aps_final, cost_centre_lookup_aps, on="Cost Centre", how="left")

aps_final2["Cost Centre Error Flag"] = np.where(aps_final2["Reporting"] == "Valid", "Valid", 
                                                         np.where(aps_final2["Reporting"].isna(), "Missing", "Invalid"))

In [ ]:
aps_final2["Reporting"].unique()

In [ ]:
print(len(aps_final2[aps_final2["Cost Centre Error Flag"] =="Missing"]),  
      len(aps_final2[aps_final2["Cost Centre Error Flag"] =="Invalid"]),
      len(aps_final2[aps_final2["Cost Centre Error Flag"] =="Valid"]),
      len(aps_final2["Cost Centre Error Flag"]))

In [ ]:
#HR LOCATION
aps_final3 = pd.merge(aps_final2, hr_location_lookup_aps, on="HR Location", how="left")

aps_final3["HR Location Error Flag"] = np.where(aps_final3["HR Location"].isnull(), "Missing", 
                                                         np.where(aps_final3["Reporting.3"] == "Valid", "Valid", "Invalid"))

In [ ]:
print(len(aps_final3[aps_final3["HR Location Error Flag"] =="Missing"]),  
      len(aps_final3[aps_final3["HR Location Error Flag"] =="Invalid"]),
      len(aps_final3[aps_final3["HR Location Error Flag"] =="Valid"]),
      len(aps_final3["HR Location Error Flag"]))

In [ ]:
#INTERNAL FUNCTION
aps_final4 = pd.merge(aps_final3, internal_function_lookup_aps, on="Internal Function", how="left")

aps_final4["Internal Function Error Flag"] = np.where(aps_final4["Internal Function"].isnull(), "Missing", 
                                                         np.where(aps_final4["Reporting.1"] == "Valid", "Valid", "Invalid"))

In [ ]:
print(len(aps_final4[aps_final4["Internal Function Error Flag"] =="Missing"]),  
      len(aps_final4[aps_final4["Internal Function Error Flag"] =="Invalid"]),
      len(aps_final4[aps_final4["Internal Function Error Flag"] =="Valid"]),
      len(aps_final4["Internal Function Error Flag"]))

In [ ]:
#JOB
aps_final5 = pd.merge(aps_final4, job_title_lookup_aps, on="Job", how="left")

aps_final5["Job Error Flag"] = np.where(aps_final5["Job"].isnull(), "Missing",
                                        np.where(aps_final5["Reporting.2"] == "Valid", "Valid", "Invalid"))

In [ ]:
print(len(aps_final5[aps_final5["Job Error Flag"] =="Missing"]),  
      len(aps_final5[aps_final5["Job Error Flag"] =="Invalid"]),
      len(aps_final5[aps_final5["Job Error Flag"] =="Valid"]),
      len(aps_final5["Job Error Flag"]))

In [ ]:
aps_final_report = aps_final5[['Employee Number', 'Cost Centre', 'Establishment', 'HR Location', 'Internal Function', 
                              'Grade', 'Job', 'FTE', 'Location Post Code', 'Area_Directorate', 'Hours', 
                              'Cost Centre Error Flag', 'HR Location Error Flag',
                              'Internal Function Error Flag', 'Job Error Flag']]

In [ ]:
len(aps_final_report)

In [ ]:
# Save all tables in single excel file
with io.BytesIO() as output:
    with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
        table1_final.to_excel(writer, sheet_name='Table 1')
        table2_final.to_excel(writer, sheet_name='Table 2')
        table3_final.to_excel(writer, sheet_name='Table 3')
        table4_final.to_excel(writer, sheet_name='Table 4')
        wfpt_data.to_excel(writer, sheet_name="WfPT data")
        aps_final_report.to_excel(writer, sheet_name = "AP data")
        #trend_data.to_excel(writer, sheet_name="Trend Data")    ---> To be completed
    data = output.getvalue()
s3 = boto3.resource('s3')
final_file_location = "sopdq_AV/outputs/WfPT Error Report Raw Tables " + date + ".xlsx"
s3.Bucket('alpha-piatool').put_object(Key=final_file_location, Body=data)


In [ ]:
####################################################################################################
#Transfer outputs to Excel Workbook template inputs
####################################################################################################

In [ ]:
from io import BytesIO
from tempfile import NamedTemporaryFile
import boto3
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

#Choose destination filename
#dataframe = table1_final
dest_filename = "sopdq_AV/outputs/WfPT Error Report " + date +".xlsx"

# Load Template from S3
bucket_name="alpha-piatool"
object_key="sopdq_AV/inputs/WfPT_Error_Reports - Template v6.xlsx"
bucket_object = boto3.resource('s3').Bucket(bucket_name).Object(object_key)
content = bucket_object.get()['Body'].read()

# Load Workbook and get sheet names
wb = load_workbook(filename=(BytesIO(content)))
ws_notes = wb.worksheets[0] # Notes
ws0 = wb.worksheets[1] # Trends % valid
ws1 = wb.worksheets[2] # Tables % valid
ws2 = wb.worksheets[3] # Tables % complete
ws3 = wb.worksheets[4] # WfPT data
ws4 = wb.worksheets[5] # AP data

In [ ]:
# Change dates in template
# Paste WFPT and AP data to relevant sheets
# Update trend table
# Update graph

In [ ]:
#Complete Table 1
for number in range(0, len(table1_final.reset_index())): #number of rows
    for i in range(0, len(table1_final.reset_index().values.tolist()[number])): # number of columns
        e=ws1.cell(row= 12+number, column=1+i)
        e.value=table1_final.reset_index().values.tolist()[number][i]      

In [ ]:
#Complete Table 2
for number in range(0, len(table2_final.reset_index().values.tolist())):
    for i in range(0, len(table2_final.reset_index().values.tolist()[number])):
        e=ws1.cell(row= 34+number, column=1+i)
        e.value=table2_final.reset_index().values.tolist()[number][i]

In [ ]:
#Complete Table 3
for number in range(0, len(table3_final.reset_index())):
    for i in range(0, len(table3_final.reset_index().values.tolist()[number])):
        e=ws2.cell(row= 13+number, column=1+i)
        e.value=table3_final.reset_index().values.tolist()[number][i]      

In [ ]:
#Complete Table 4
for number in range(0, len(table4_final.reset_index().values.tolist())):
    for i in range(0, len(table4_final.reset_index().values.tolist()[number])):
        e=ws2.cell(row= 35+number, column=1+i)
        e.value=table4_final.reset_index().values.tolist()[number][i]

In [ ]:
# #Add WFPT_data and AP_final data
# for number in range(0, len(a.reset_index().values.tolist())):
#     for i in range(0, len(a.reset_index().values.tolist()[number])):
#         e=ws3.cell(row= 9+number, column=1+i)
#         e.value=a.reset_index().values.tolist()[number][i]

In [ ]:
# Save Workbook back to S3
s3 = boto3.client('s3')
with NamedTemporaryFile() as tmp:
    filename = '/tmp/{}'.format("WfPT_Error_Reports - Template v6.xlsx")
    wb.save(filename)
    wfpt_data.to_excel(writer, sheet_name='WfPT data', startrow = 9, startcol = 1)
    s3.upload_file(Bucket =bucket_name, Filename = filename, Key=dest_filename)
        
        

In [ ]:
# # Save all lookups in single excel file
# with io.BytesIO() as output:
#     with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
#         cost_centre_lookup.to_excel(writer, sheet_name='Cost Centre Lookup')
#         internal_function_lookup.to_excel(writer, sheet_name='Internal Function Lookup')
#         hr_location_lookup.to_excel(writer, sheet_name='HR Location Lookup')
#         job_title_lookup.to_excel(writer, sheet_name='Job Title Lookup')
#         cost_centre_consistency.to_excel(writer, sheet_name="Cost Centre type")
#     data = output.getvalue()
# s3 = boto3.resource('s3')
# s3.Bucket('alpha-piatool').put_object(Key='sopdq_AV/outputs/WFPT lookups.xlsx', Body=data)
